In [1]:
import json
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter
from time import time

In [44]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
# pd.set_option('max_colwidth', 200)

In [3]:
pickle_in = open(r"C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Review_features_data.pickle", 'rb')
df = pickle.load(pickle_in)
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,22,21,228544.0
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1,10,9,228544.0
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,22,21,228544.0
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,5,5,228544.0
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,12,12,228544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041164,5,True,"10 3, 2016",A3Q7GA2Q62E18Q,B01HE0RNYO,NaN,Kerry Moore,Excellent product for the price!!!,Inexpensive but good quality...,1475452800,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,3,3,574746.0
1041165,5,False,"07 18, 2016",AEKOIX49JGCUG,B01HE0RNYO,NaN,Wishart,I needed a new phone case for my iPhone 6 plus...,Great Case. Gonna Buy Again.,1468800000,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,24,152,114,574746.0
1041166,4,False,"07 16, 2016",A2KOKSKEDIBRGQ,B01HE0RNYO,NaN,Wise One,The covers my iPhone just fine. It is simple i...,Plain Jane of iPhone Covers,1468627200,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,28,233,159,574746.0
1041167,3,False,"07 11, 2016",A1BO3NH4KKASUT,B01HE0RNYO,NaN,Bebescastle,This free case is okay it sli

In [4]:
rows = list(df.index)

## Getting rank of 2nd element in 'rank' list and making subcategories

In [5]:
a = df.iloc[0]['rank']
a

"['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']"

In [6]:
# Ref - https://stackoverflow.com/questions/24890779/python-list-inside-string-to-list
a = eval(a)
a

['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)',
 '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']

In [7]:
a = a[1]
a

'>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips'

In [8]:
def rank_calc(rank_str):
    rank = ''
    for i in range(len(rank_str)):
        if rank_str[i].isdigit() == True:
            rank += rank_str[i]
        elif rank_str[i] == 'i':
            break
    return rank

In [9]:
rank_calc(a)

'114920'

In [10]:
rank_list1 = [rank_calc(eval(df.iloc[i]['rank'])[1]) for i in range(343534,343544)]
rank_list1

['33078',
 '33078',
 '33078',
 '142956',
 '142956',
 '142956',
 '142956',
 '142956',
 '142956',
 '142956']

### Some rank rows have nothing in them (i.e. []), which makes it impossible to run the code which takes the 1st index (2nd element) of the list, as there is nothing there, so dropping them.

In [11]:
df[df['rank'] == '[]']

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no
530,4,False,"01 22, 2005",A1TTKKN4AY63QK,B00005UK9X,{'Package Type:': ' Standard Packaging'},Jan R. Klincewicz,"I drive a company-supplied vehicle, so alterat...",Rocks for normal drivers,1106352000,26,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,62,54,NaN
531,5,True,"06 25, 2003",A16PKR0PIB6959,B00005UK9X,{'Package Type:': ' Standard Packaging'},David Levine,I have a GPS V that I really like and this is ...,Great add-on,1056499200,14,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,62,52,NaN
532,5,True,"11 11, 2012",A1V4KTY1RXI606,B00005UK9X,{'Package Type:': ' Standard Packaging'},Barbara P.,As long as there are products like this availa...,A real grabber,1352592000,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,20,19,NaN
533,5,True,"08 31, 2009",A19RGDJPNZ7NZ,B00005UK9X,{'Package Type:': ' Standard Packaging'},MW,"I use it with a Garmin StreetPilot 7200, which...",Works with Garmin 7200,1251676800,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,82,71,NaN
534,5,True,"04 4, 2008",A100WO06OQR8BQ,B00005UK9X,{'Package Type:': ' Standard Packaging'},Duffer.,Works great. Easy to attach the GPS and easy t...,Great Device,1207267200,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,13,12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041154,1,True,"08 11, 2016",AKJK5FH23L8ZA,B01H72UJZ4,NaN,Rodolfo Tarelli,Over one milimiter uncovered in all the temper...,Really Bad Fit,1470873600,0,['https://images-na.ssl-images-amazon.com/imag...,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,5,22,20,NaN
1041155,2,False,"08 3, 2016",A3TZH1A8PYUK1O,B01H72UJZ4,NaN,TuckerReviewer,"This is a clear glass protector, theoretically...",Not sized correctly.,1470182400,2,['https://images-na.ssl-images-amazon.com/imag...,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,9,57,47,NaN
1041156,4,False,"08 2, 2016",ARKQFG8F2FJ8L,B01H72UJZ4,NaN,Amazon Customer,I've used quite a few screen protectors over y...,"Durable, a little narrow",1470096000,0,NaN,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,19,137,108,NaN
1041157,3,False,"07 20, 2016",A24BH8L76QQFVD,B01H72UJZ4,NaN,PastorONeil,This is a tempered glass screen protector for ...,This is a bit annoying as dust and other parti...,1468972800,0,NaN,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,17,157,101,NaN


In [12]:
nothing_list = [i for i in rows if df.iloc[i]['rank'] == '[]']

In [13]:
nothing_list

[530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 590,
 591,
 592,
 593,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 730,
 731,
 732,
 733,
 734,
 735,
 736,
 737,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 760,
 761,
 762,
 763,
 764,
 765,
 766,
 767,
 768,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 778,
 779,
 780,
 781,
 782,
 783,
 784,
 785,
 786,
 787,
 788,
 789,
 790,
 791

In [14]:
len(nothing_list)

66796

In [15]:
df.drop(nothing_list, inplace=True)

In [16]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

### In rank column, some rows do not follow the usual format (they are starting with, for eg., 1,420,411 having no follow up rank and subcategories), so removing them.

In [17]:
drop_list = [i for i in rows if df.iloc[i]['rank'][0].isdigit() == True]

In [18]:
drop_list

[589,
 590,
 591,
 592,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 600,
 1608,
 1609,
 1610,
 1611,
 1612,
 1613,
 3309,
 3310,
 3311,
 3312,
 3313,
 3314,
 7572,
 7573,
 7574,
 7575,
 7576,
 7577,
 7578,
 7806,
 7807,
 7808,
 7809,
 7810,
 7811,
 7812,
 7813,
 7814,
 7815,
 7816,
 7817,
 7818,
 7819,
 11090,
 11091,
 11092,
 11093,
 11094,
 11095,
 11096,
 11097,
 11098,
 11099,
 11100,
 11101,
 11102,
 11103,
 11104,
 13102,
 13103,
 13104,
 13105,
 13106,
 13107,
 13108,
 13109,
 13110,
 13111,
 13112,
 13113,
 13114,
 13115,
 13116,
 13117,
 13118,
 13119,
 16575,
 16576,
 16577,
 16578,
 16579,
 16580,
 16581,
 19171,
 19172,
 19173,
 19174,
 19175,
 19176,
 21602,
 21603,
 21604,
 21605,
 21606,
 21607,
 21608,
 21609,
 23535,
 23536,
 23537,
 23538,
 23539,
 23540,
 23541,
 23542,
 23604,
 23605,
 23606,
 23607,
 23608,
 23609,
 23610,
 23611,
 23612,
 23613,
 23614,
 23615,
 23616,
 23617,
 23944,
 23945,
 23946,
 23947,
 23948,
 23949,
 23950,
 23951,
 23952,
 23953,
 23954,
 

In [19]:
len(drop_list)

9292

In [20]:
df.drop(drop_list, inplace=True)

In [21]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

### More things

In [22]:
one_list = [i for i in rows if len(eval(df.iloc[i]['rank'])) == 1]

In [23]:
one_list

[4854,
 4855,
 4856,
 4857,
 4858,
 4859,
 4860,
 4861,
 4862,
 4863,
 4864,
 4865,
 4866,
 4867,
 4868,
 4869,
 4870,
 4871,
 4872,
 4873,
 4874,
 4875,
 4876,
 4877,
 4878,
 4879,
 4880,
 4881,
 4882,
 4883,
 4884,
 4885,
 4886,
 4887,
 4888,
 4889,
 4890,
 4891,
 4892,
 4893,
 4894,
 4895,
 4896,
 4897,
 4898,
 4899,
 4900,
 4901,
 4902,
 4903,
 4904,
 4905,
 4906,
 4907,
 4908,
 4909,
 4910,
 4911,
 4912,
 4913,
 4914,
 4915,
 4916,
 4917,
 4918,
 4919,
 4920,
 4921,
 4922,
 4923,
 4924,
 4925,
 4926,
 4927,
 4928,
 4929,
 4930,
 4931,
 4932,
 4933,
 4934,
 4935,
 4936,
 4937,
 4938,
 4939,
 4940,
 4941,
 4942,
 4943,
 4944,
 4945,
 4946,
 4947,
 4948,
 4949,
 4950,
 4951,
 4952,
 4953,
 4954,
 4955,
 4956,
 4957,
 4958,
 4959,
 4960,
 4961,
 4962,
 4963,
 4964,
 4965,
 4966,
 4967,
 4968,
 4969,
 4970,
 4971,
 4972,
 4973,
 4974,
 4975,
 4976,
 4977,
 4978,
 4979,
 4980,
 4981,
 4982,
 4983,
 4984,
 4985,
 4986,
 4987,
 4988,
 4989,
 4990,
 4991,
 4992,
 4993,
 4994,
 4995,
 4996,

In [46]:
len(one_list)

2192

In [47]:
df.drop(one_list, inplace=True)

In [48]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

In [49]:
start = time()

In [51]:
rank_list = [rank_calc(eval(df.iloc[i]['rank'])[1]) for i in rows]

In [52]:
end = time()
end - start

177.4822952747345

In [53]:
rank_list

['114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '114920',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '275153',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',
 '165573',

## Sub-categories

In [54]:
trial = [len(eval(df.iloc[i]['rank'])[1].split('>')) for i in rows]

In [55]:
trial

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,


In [56]:
min(trial)

3

In [57]:
max(trial)

5

In [58]:
set(trial)

{3, 4, 5}

In [59]:
e = eval(df.iloc[0]['rank'])
e

['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)',
 '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']

In [60]:
e = e[1]
e

'>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips'

In [61]:
e = e.split('>')
e

['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips']

In [62]:
e[2]

' Cases, Holsters & Clips'

In [63]:
f = eval(df.iloc[1]['rank'])[1].split('>')
f[2]

' Cases, Holsters & Clips'

In [64]:
sub_cat_2nd_ele_list = [  eval(df.iloc[i]['rank'])[1].split('>') for i in rows  ]

In [65]:
sub_cat_2nd_ele_list

[['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],
 ['', '#114,920 in Cell Phones & Accessories ', ' Cases, Holsters & Clips'],

In [66]:
sub_cat_1_list = [  sub_cat_2nd_ele_list[i][2] for i in rows ]  # no need to write if (trial[i] == 3 or trial[i] == 4 or trial[i] == 5) else np.nan, it would be redundant

In [67]:
sub_cat_1_list

[' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Holsters & Clips',
 ' Cases, Hols

In [68]:
sub_cat_2_list = [  sub_cat_2nd_ele_list[i][3] if (trial[i] == 4 or trial[i] == 5) else np.nan for i in rows ]

In [69]:
sub_cat_2_list

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Basic Cases',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',
 ' Chargers & Power Adapters',


In [70]:
sub_cat_3_list = [  sub_cat_2nd_ele_list[i][4] if trial[i] == 5 else np.nan for i in rows ]

In [71]:
sub_cat_3_list

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' Car Chargers',
 ' C

In [72]:
df['sub_cat_1'] = sub_cat_1_list
df['sub_cat_2'] = sub_cat_2_list
df['sub_cat_3'] = sub_cat_3_list

In [73]:
df['sub_cat_rank'] = rank_list

In [74]:
df.head(700)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_rank
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,22,21,228544.0,"Cases, Holsters & Clips",NaN,NaN,114920
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1,10,9,228544.0,"Cases, Holsters & Clips",NaN,NaN,114920
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,22,21,228544.0,"Cases, Holsters & Clips",NaN,NaN,114920
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,5,5,228544.0,"Cases, Holsters & Clips",NaN,NaN,114920
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,12,12,228544.0,"Cases, Holsters & Clips",NaN,NaN,114920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,5,True,"07 8, 2012",A12NANE7QGSGY3,B0002KRC5Y,NaN,Emerald Burgos,this was a great addition to the boat especial...,simple but very useful,1341705600,0,NaN,['This 12 volt receptacle is manufactured from...,['Interlocking connection system provides secu...,"['>#16,685 in Cell Phones & Accessories (See T...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,8,8,16685.0,Cell Phone Accessories,NaN,NaN,10773
696,5,True,"06 13, 2012",A3UVA4Q05TZZW,B0002KRC5Y,NaN,paboothz71,I am using this connector as charger for my po...,Works great.,1339545600,0,NaN,['This 12 volt receptacle is manufactured from...,['Interlocking connection system provides secu...,"['>#16,685 in Cell Phones & Accessories (See T...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,11,11,16685.0,Cell Phone Accessories,NaN,NaN,10773
697,4,False,"08 23, 2010",A2V1J3JT5OOZFO,B0002KRC5Y,NaN,A Long-time Customer,Overall it is fine for now. I installed it in ...,"Nice little device, but...",1282521600,0,NaN,['This 12 volt receptacle is manufactured from...,['Interlocking conne

In [77]:
"""writer = pd.ExcelWriter(r"C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Review_features_data_1.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer, index=False)
writer.close()"""